In [11]:
import os
from os import listdir
from os.path import isfile, isdir, join
import json
import re

In [21]:
def load_all_files(path):
    for file in listdir(path):
     filePath = join(path,file)
     if isfile(filePath):
        yield filePath
     elif isdir(filePath):
        yield from load_all_files(filePath)

def makedir_if_not_exists(filename: str) -> None:
    try:
        os.makedirs(filename)
    except:
        pass

In [24]:
pattern = re.compile(r'(?<!^)(?=[A-Z])')

def fix_key(k):
    return pattern.sub('_', k).lower()

def fix_element(e):
    if isinstance(e, dict):
        return {fix_key(key): fix_element(value) for key, value in e.items()}
    else:
        return e

def fix_file(source_dir, destination_dir, file):
    with open(file) as f:
        data = json.load(f)
        fixed = fix_element(data)
        relative_path = os.path.relpath(file, source_dir)
        dest_path = os.path.join(destination_dir, relative_path)
        makedir_if_not_exists(os.path.dirname(dest_path))
        with open(dest_path, 'w') as outfile:
            json.dump(fixed, outfile)
        
        

In [25]:
source_dir = '/home/ctripp/log/'
dest_dir = '/home/ctripp/log2/'
for file in load_all_files(source_dir):
    fix_file(source_dir, dest_dir, file)
    